In [8]:
using Distributions

struct TwoBinomials{T<:Real} <: DiscreteMultivariateDistribution
    m::Int
    p::T
    n::Int
    q::T
end

Base.length(d::TwoBinomials) = 4
Base.eltype(d::TwoBinomials) = Int
Distributions.params(d::TwoBinomials) = (d.m, d.p, d.n, d.q)

function Distributions._rand!(rng::Distributions.AbstractRNG, d::TwoBinomials, x::AbstractVector)
    k = rand(rng, Binomial(d.m, d.p))
    l = rand(rng, Binomial(d.n, d.q))
    @inbounds x[1], x[2], x[3], x[4] = k, l, d.m - k, d.n - l
    x
end

function Distributions._rand!(rng::Distributions.AbstractRNG, d::TwoBinomials, x::AbstractMatrix)
    @inbounds for j in axes(x, 2) 
        k = rand(rng, Binomial(d.m, d.p))
        l = rand(rng, Binomial(d.n, d.q))
        x[1,j], x[2,j], x[3,j], x[4,j] = k, l, d.m - k, d.n - l
    end
    x
end

function Distributions._logpdf(d::TwoBinomials, x::AbstractVector)
    x[1] + x[3] == d.m || x[2] + x[4] == d.m || return -Inf
    logpdf(Binomial(d.m, d.p), x[1]) + logpdf(Binomial(d.n, d.q), x[2]) 
end

Distributions.mean(d::TwoBinomials) = [d.m*d.p, d.n*d.q, d.m*(1 - d.p), d.n*(1 - d.q)]

In [2]:
d = TwoBinomials(10, 0.25, 20, 0.75)

TwoBinomials{Float64}(m=10, p=0.25, n=20, q=0.75)

In [9]:
params(d)

(10, 0.25, 20, 0.75)

In [3]:
rand(d)

4-element Vector{Int64}:
  3
 15
  7
  5

In [4]:
rand(d, 16)

4×16 Matrix{Int64}:
  3   4   5   0   3   1   3   2   2   3   5   4   1   2   4   1
 13  15  17  17  17  14  17  13  14  17  15  15  11  16  17  17
  7   6   5  10   7   9   7   8   8   7   5   6   9   8   6   9
  7   5   3   3   3   6   3   7   6   3   5   5   9   4   3   3

In [5]:
pdf(d, [3, 16, d.m-3, d.n-16])

0.04747490956783132

In [6]:
mean(d)

4-element Vector{Float64}:
  2.5
 15.0
  7.5
  5.0